In [1]:
from dotenv import load_dotenv

load_dotenv('../../.env')

True

# Didn't Work

In [5]:
"""
    1. No Custom Knowledge Base
    2. Naively Splitting the Custom Knowledge Base
"""

# Smart Splitting

In [2]:
text = """
Objection: "There's no money."
It could be that your prospect's business simply isn't big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.

Objection: "We don't have any budget left this year."
A variation of the "no money" objection, what your prospect's telling you here is that they're having cash flow issues. But if there's a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.

Objection: "We need to use that budget somewhere else."
Prospects sometimes try to earmark resources for other uses. It's your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a massive ROI with you.
"""

# Split the text into a list using the keyword "Objection: "
objections_list = text.split("Objection: ")[1:]  # We ignore the first split as it is empty

# Now, prepend "Objection: " to each item as splitting removed it
objections_list = ["Objection: " + objection for objection in objections_list]

In [3]:
objections_list

['Objection: "There\'s no money."\nIt could be that your prospect\'s business simply isn\'t big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.\n\n',
 'Objection: "We don\'t have any budget left this year."\nA variation of the "no money" objection, what your prospect\'s telling you here is that they\'re having cash flow issues. But if there\'s a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.\n\n',
 'Objection: "We need to use that budget somewhere else."\nProspects sometimes try to earmark resources for other uses. It\'s your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a

# Integrating SalesCopilot with Deep Lake

### Creating, Loading, and Querying Our Database for AI

In [10]:
import os
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import DeepLake


embeddings = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2',
                                       model_kwargs = {'device':'cpu'} )
class DeepLakeLoader:
    def __init__(self, source_data_path):
        """
        Initialize DeepLakeLoader object.

        Args:
            source_data_path (str): Path to the source data file. Should be a text file.
        """
        self.source_data_path = source_data_path
        self.file_name = os.path.basename(source_data_path)
        self.data = self.split_data()

        if self.check_if_db_exists():
            self.db = self.load_db()
        else:
            self.db = self.create_db()

    def check_if_db_exists(self):
        """
        Check if the database already exists.

        Returns:
            bool: True if the database exists, False otherwise.
        """
        return os.path.exists(f'deeplake/{self.file_name}')

    def load_db(self):
        """
        Load the database if it already exists.

        Returns:
            DeepLake: DeepLake object.
        """
        return DeepLake(dataset_path=f'deeplake/{self.file_name}', embedding_function=embeddings, read_only=True)

    def create_db(self):
        """
        Create the database if it does not already exist.

        Databases are stored in the deeplake directory.

        Returns:
            DeepLake: DeepLake object.
        """
        return DeepLake.from_texts(self.data, embeddings, dataset_path=f'deeplake/{self.file_name}')

    def query_db(self, query):
        """
        Query the database for passages that are similar to the query.

        Args:
            query (str): Query string.

        Returns:
            content (list): List of passages that are similar to the query.
        """
        results = self.db.similarity_search(query, k=3)
        content = []
        for result in results:
            content.append(result.page_content)
        return content

    def split_data(self):
        """
        Preprocess the data by splitting it into passages.

        If using a different data source, this function will need to be modified.

        Returns:
            split_data (list): List of passages.
        """
        with open(self.source_data_path, 'r') as f:
            content = f.read()
        split_data = re.split(r'(?=\d+\. )', content) # This is super specific to the default data source! If using a different data source, this will need to be modified.
        if split_data[0] == '':
            split_data.pop(0)
        split_data = [entry for entry in split_data if len(entry) >= 30]
        return split_data

In [11]:
db = DeepLakeLoader('data/salestexting.txt')

Creating 44 embeddings in 1 batches of size 44:: 100%|██████████| 1/1 [00:05<00:00,  5.37s/it]

Dataset(path='deeplake/salestexting.txt', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype      shape     dtype  compression
  -------    -------    -------   -------  ------- 
   text       text      (44, 1)     str     None   
 metadata     json      (44, 1)     str     None   
 embedding  embedding  (44, 384)  float32   None   
    id        text      (44, 1)     str     None   


In [12]:
LIVE_CHAT_PROMPT = """
Reminder: You're SalesCopilot.
Your goal is to help the user in their sales call with the customer. 
Using conversation transcripts, you'll help create responses and guide the user (labeled You).
Keep your responses helpful, concise, and relevant to the conversation.  
The transcripts may be fragmented, incomplete, or even incorrect. Do not ask for clarification, do your best to understand what
the transcripts say based on context. Be sure of everything you say.
Keep responses concise and to the point. Starting now, answer the user's question based on the transcript:

"""

DETECT_OBJECTION_PROMPT = """
Your task is to read the transcript and discern whether the customer is raising any objections to the product or service the salesperson is selling.
If the customer is simply stating their thoughts, preferences, or facts that are not specifically connected to the product or service, it is not an objection. 
Quote only from the transcript.
Do not add, infer, or interpret anything.
Example:
'''
Customer: I'm not sure if I can afford this. It's a bit expensive. The sky is blue. I like the color blue. 

You: I'm not sure if I can afford this. It's a bit expensive.
'''
If there is no objection, respond with 'None'.
Starting now, you will respond only with either the quote or None: 
"""

OBJECTION_GUIDELINES_PROMPT = """
You are SalesCopilot. You will be provided with a customer objection, and a selection
of guidelines on how to respond to certain objections. 
Using the provided content, write out the objection and the actionable course of action you recommend.
Objections sound like:
'''It's too expensive.
There's no money.
We don't have any budget left.
I need to use this budget somewhere else.
I don't want to get stuck in a contract.
We're already working with another vendor.
I'm locked into a contract with a competitor.
I can get a cheaper version somewhere else.'''
 
Example of your message:

'It seems like the customer is {explain their objection}.

I recommend you {course of action for salesperson}.'

"""

SAVED_TRANSCRIPT_PROMPT ="""
You are SalesCopilot. You will be provided with a transcript of a sales call between the user and a customer.
Answer any questions the user asks you. You may also assess the user's performance and provide feedback.
The transcripts may be fragmented, incomplete, or even incorrect. Do not ask for clarification, do your best to understand what
the transcripts say based on context.
The speaker labeled "You" in the transcripts is the user you are helping.
"""

In [13]:
results = db.query_db(DETECT_OBJECTION_PROMPT)

# Use Chat Models

In [18]:
from langchain import HuggingFaceHub

llm_mistral = HuggingFaceHub(
    repo_id='mistralai/Mistral-7B-Instruct-v0.2',
    model_kwargs={'temperature':0.5,"max_length": 64,"max_new_tokens":512}
)

c:\Users\Ghost\anaconda3\envs\genai360\lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'InferenceApi' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [19]:
from langchain_community.chat_models.huggingface import ChatHuggingFace

chat_mistral = ChatHuggingFace(llm=llm_mistral)

WARNING! repo_id is not default parameter.
                    repo_id was transferred to model_kwargs.
                    Please confirm that repo_id is what you intended.
WARNING! task is not default parameter.
                    task was transferred to model_kwargs.
                    Please confirm that task is what you intended.
WARNING! huggingfacehub_api_token is not default parameter.
                    huggingfacehub_api_token was transferred to model_kwargs.
                    Please confirm that huggingfacehub_api_token is what you intended.


In [23]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

starting_message = HumanMessage(content="Hello!!",role='user')
system_message = AIMessage(content=OBJECTION_GUIDELINES_PROMPT,role='assistant')
human_message = HumanMessage(content=f'Customer objection: {DETECT_OBJECTION_PROMPT} | Relevant guidelines: {results}', role='user')

response = chat_mistral([starting_message, system_message, human_message])

In [25]:
print(response.content)

<s>[INST] Hello!! [/INST]
You are SalesCopilot. You will be provided with a customer objection, and a selection
of guidelines on how to respond to certain objections. 
Using the provided content, write out the objection and the actionable course of action you recommend.
Objections sound like:
'''It's too expensive.
There's no money.
We don't have any budget left.
I need to use this budget somewhere else.
I don't want to get stuck in a contract.
We're already working with another vendor.
I'm locked into a contract with a competitor.
I can get a cheaper version somewhere else.'''
 
Example of your message:

'It seems like the customer is {explain their objection}.

I recommend you {course of action for salesperson}.'

</s>[INST] Customer objection: 
Your task is to read the transcript and discern whether the customer is raising any objections to the product or service the salesperson is selling.
If the customer is simply stating their thoughts, preferences, or facts that are not specific